# Credit Risk Resampling Techniques

In [193]:
import warnings
warnings.filterwarnings('ignore')

In [194]:
import numpy as np
import pandas as pd
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import balanced_accuracy_score

from imblearn.metrics import classification_report_imbalanced
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTE

# Read the CSV into DataFrame

In [195]:
# Load the data
lending_data = pd.read_csv('./Resources/lending_data.csv')
lending_data.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [196]:
# Create our features
X = lending_data.drop(columns='loan_status')
X = pd.get_dummies(X, columns=['homeowner'])

# Create our target
y = lending_data['loan_status'].to_frame()
encoder = LabelEncoder()
encoder.fit(y['loan_status'])
y_encoded = encoder.transform(y['loan_status'])
y['loan_status'] = y_encoded

In [197]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [198]:
# Check the balance of our target values
y.value_counts()

loan_status
1              75036
0               2500
dtype: int64

In [199]:
# Create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    random_state=1,
)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [200]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)

In [201]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [202]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [203]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9889115309798473

In [204]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  609,    10],
       [  113, 18652]])

In [205]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.84      0.98      0.99      0.91      0.99      0.98       619
          1       1.00      0.99      0.98      1.00      0.99      0.98     18765

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [206]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_train_ros, y_train_ros = ros.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_train_ros['loan_status'])

Counter({1: 56271, 0: 56271})

In [207]:
# Train the Logistic Regression model using the resampled data
model.fit(X_train_ros, y_train_ros)
y_ros_pred = model.predict(X_test)

In [208]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_ros_pred)

0.027238478468705918

In [209]:
# Display the confusion matrix
confusion_matrix(y_test, y_ros_pred)

array([[    4,   615],
       [17864,   901]])

In [210]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_ros_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.00      0.01      0.05      0.00      0.02      0.00       619
          1       0.59      0.05      0.01      0.09      0.02      0.00     18765

avg / total       0.58      0.05      0.01      0.09      0.02      0.00     19384



### SMOTE Oversampling

In [211]:
# Resample the training data with SMOTE
smote = SMOTE(random_state=1)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_train_smote['loan_status'])

Counter({1: 56271, 0: 56271})

In [212]:
# Train the Logistic Regression model using the resampled data
model.fit(X_train_smote, y_train_smote)
y_smote_pred = model.predict(X_test)

In [213]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_smote_pred)


0.5

In [214]:
# Display the confusion matrix
confusion_matrix(y_test, y_smote_pred)


array([[  619,     0],
       [18765,     0]])

In [191]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_smote_pred))


                   pre       rec       spe        f1       geo       iba       sup

          0       0.03      1.00      0.00      0.06      0.00      0.00       619
          1       0.00      0.00      1.00      0.00      0.00      0.00     18765

avg / total       0.00      0.03      0.97      0.00      0.00      0.00     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [220]:
# Resample the data using the ClusterCentroids resampler
cc = ClusterCentroids(random_state=1)
X_cc, y_cc = cc.fit_resample(X_train_scaled, y_train)
# View the count of target classes with Counter
Counter(y_cc)

KeyboardInterrupt: 

In [ ]:
# Train the Logistic Regression model using the resampled data


In [ ]:
# Calculate the balanced accuracy score


In [ ]:
# Display the confusion matrix


In [ ]:
# Print the imbalanced classification report


# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [ ]:
# Resample the training data with SMOTEENN


# View the count of target classes with Counter


In [ ]:
# Train the Logistic Regression model using the resampled data


In [ ]:
# Calculate the balanced accuracy score


In [ ]:
# Display the confusion matrix


In [ ]:
# Print the imbalanced classification report


# Final Questions

1. Which model had the best balanced accuracy score?

   YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.
